In [1]:
from konlpy.tag import *
import datetime
from IPython.display import display

import re
import numpy as np
import pandas as pd
from pprint import pprint
from konlpy.tag import Hannanum
from konlpy.tag import Twitter
twitter = Okt()
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

from pykospacing import spacing
import warnings
from IPython.display import display , Markdown

Using TensorFlow backend.


- There is no widely known and used stopwords packages for korean.
- Therefore I downloaded the csv from some webpage and read for pandas read_csv method.

In [2]:
stop_words = list(pd.read_csv('kor_stop_words.csv').T.iloc[:1,:].values[0])
stop_words[:10]

['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라']

# Overall Tuning process is started.

In [3]:
df = pd.read_csv('concat_data.csv')
df.tail()

,click,content,time,title
9198,3190,▲ 이더리움 비트코인 비트코인과 함께 이더리움에 관심이 쏠리고 있다한 매체 보도에 ...,05-25,비트코인 이어 이더리움도 관심 스마트 콘트랙트 전자계약이 뭐길래
9199,2706,비트코인 © AFP뉴스1대표적인 전자화폐 비트코인의 랠리가 눈부시다 올 들어서만 그...,05-25,비트코인보다 눈부신 이더리움…올해 들어 2300 급등
9200,3067,Photo ⓒpixabay▲비트코인Bitcoin이 화제다 비트코인은 2009년 나...,05-25,비트코인 이더리움 가상화폐 수요 급증 이유는
9201,2801,사진게티이미지뱅크 비트코인 가상화폐비트코인 이더리움 등 가상화폐의 가격이 계속해서 ...,05-25,몸값 오르는 가상화폐…비트코인·이더리움 뭐길래
9202,2341,비트코인이 2000달러를 돌파한지 닷새만에 또다시 최고치를 새로 썼다고 24일현지시...,05-25,비트코인 2500달러 또 사상최고치


first-tuning

In [4]:
content_ls = df['content'].tolist()
# remove korean emotional emoji
content_ls = [i.replace('ㅜ','').replace('ㅠ','').replace('ㅋ','').replace('ㅎ','') if type(i) == str else np.nan for i in content_ls] 
# eliminate etc words such as ▲
content_ls = [','.join(re.findall('[가-힣a-zA-Z0-9\s]',i)).replace(',','') if type(i) == str else np.nan for i in content_ls]
df['content'] = content_ls

nan-value eliminating and make new dataframe named new_df

In [5]:
new_df = df[df['content'].notnull()].reset_index(drop=True)

- some content hasn't any spacing in their sentences. I thought they might have some problem to tune and interpret 
- Therefore, I remove them and make new dataframe named tuned_df

In [6]:
new_idx = sorted(list(set(new_df.index).difference(set([idx for idx,val in enumerate(new_df['content']) if ' ' not in val]))))

In [7]:
tuned_df = new_df.loc[new_idx]

In [8]:
df.shape , new_df.shape , tuned_df.shape

((9203, 4), (9179, 4), (8441, 4))

spacing_function

In [9]:
def split_and_spacing(content):
    warnings.filterwarnings("error")
    tuning_process_1 = [i for i in content.split(" ") if 'http' not in i]
    
    tuning_process_2 = []
    for idx,i in enumerate(tuning_process_1) : 
        try : 
            tuning_process_2.append(spacing(i))
        except :
            display(Markdown("### some content's tuning gonna be passed!"))
            tuning_process_2.append(i)
            
    tuning_process_3 = ','.join(tuning_process_2).replace(',',' ')
    return tuning_process_3

In [ ]:
%time tuned_df['tuned_content'] = [split_and_spacing(i) for i in tuned_df['content'].tolist()]

### some content's tuning gonna be passed!

### some content's tuning gonna be passed!

### some content's tuning gonna be passed!

### some content's tuning gonna be passed!

### some content's tuning gonna be passed!

### some content's tuning gonna be passed!

### some content's tuning gonna be passed!

### some content's tuning gonna be passed!

### some content's tuning gonna be passed!

In [ ]:
tuned_df.to_csv('new_tuned_df.csv',index=False)

In [ ]:
%time tuned_df['tuned_content'] = tuned_df['content'].apply(lambda x : split_and_spacing(x))